In [2]:
from fake_useragent import UserAgent
from selenium import webdriver
from selenium.webdriver import Chrome
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager
import bs4
import pandas as pd
import time
import undetected_chromedriver as uc
import argparse
import random
import urllib3
import warnings
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
warnings.filterwarnings("ignore")


def random_time_delay():
    time.sleep(random.uniform(60,80))


def open_driver(): 
    ua = UserAgent()
    chrome_options = uc.ChromeOptions()
    chrome_options.add_argument("--window-size=1920,1080")
    chrome_options.add_argument("--disable-extensions")
    chrome_options.add_argument('--disable-notifications')
    chrome_options.add_argument("--mute-audio")
    chrome_options.add_argument("--start-maximized")
    chrome_options.add_argument('--disable-gpu')
    chrome_options.add_argument('--disable-dev-shm-usage')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--ignore-certificate-errors')
    #chrome_options.add_argument(f"user-agent={ua['google chrome']}")

    driver = uc.Chrome(executable_path=ChromeDriverManager().install(), options=chrome_options)
    
    return driver


def login(driver, username, password):  
    # go to login page
    driver.get("https://www.instagram.com")
    random_time_delay()

    # Type username
    emailid = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.NAME, "username"))) 
    emailid.send_keys(username)
    random_time_delay()

    # Type password
    passwordid = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.NAME, "password")))
    passwordid.send_keys(password)
    random_time_delay()

    # Click on the signin button
    signinButton = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH, '//*[@id="loginForm"]/div/div[3]/button/div')))
    ActionChains(driver).move_to_element(signinButton).click().perform()
    random_time_delay()
    
    return driver


def filter_chats(driver):
    all_chats = driver.find_elements_by_css_selector('div[class=" _ab8s _ab8w  _ab94 _ab99 _ab9f _ab9m _ab9p _abcm"]')
    print('\tTotal Chats:',len(all_chats)) 
    result = list(filter(lambda i: i.find_elements_by_css_selector('div[class="_ab8w  _ab94 _ab97 _ab9h _ab9k _ab9p  _abb0 _abcm"]')==[], all_chats)) # unseen blue dot
    print('\tSelected Chats:', len(result))
    return result



def get_last_msg(driver, chats):
    df = []
    
    for i in chats:
        try:
            # Click on the chat
            i.click()
            random_time_delay()

            # Get the username
            user = driver.find_element_by_css_selector('button[class="_acan _acao _acaq _acat"]').text
            user = user.split('\n')[0]
            
            # Get last message
            source = driver.page_source
            soup = bs4.BeautifulSoup(source, 'html.parser')
            msgs = [i.text for i in soup.find_all('div',{'class':'_aacl _aaco _aacu _aacx _aad6 _aade'})]

            # Go back to the main inbox page
            driver.back()
            random_time_delay()

            df.append([user,msgs[-1]])
            print([user,msgs[-1]])

        except Exception as e:
            print(e)
            #print('Non-text modal encountered')
            pass

        
    df2 = pd.DataFrame(df,columns = ['User','Last Message'])
    
    return df2

In [3]:
df2 = pd.read_csv('test_output_csv_files/saved_output.csv')
df2

,Channel,Saved data,Post url,Post user
0,https://www.instagram.com/melanomareportgroup,[['https://www.instagram.com/p/Bkfp6z-F6kvYxU6...,[,[
1,https://www.instagram.com/melanomareportgroup,[['https://www.instagram.com/p/BzfD8Zgl0_0AbYp...,[,[
2,https://www.instagram.com/melanomareportgroup,[['https://www.instagram.com/p/Bkfp6z-F6kvYxU6...,[,[
3,https://www.instagram.com/melanomareportgroup,[['https://www.instagram.com/p/BzfD8Zgl0_0AbYp...,[,[
4,https://www.instagram.com/guygonehostile,[['https://www.instagram.com/p/CfWUS-sPhgx9f8N...,[,[


In [14]:
import glob

In [15]:
csv_files = glob.glob(f'saved_checkpoints/*')
df2 = [pd.read_csv(i) for i in csv_files]
df2 = pd.concat(df2, ignore_index=True)
df2['Saved data'] = df2['Saved data'].apply(lambda x: eval(x))

In [16]:
df2

,Channel,Saved data
0,https://www.instagram.com/melanomareportgroup,[[https://www.instagram.com/p/Bkfp6z-F6kvYxU6s...
1,https://www.instagram.com/melanomareportgroup,[[https://www.instagram.com/p/BzfD8Zgl0_0AbYpu...
2,https://www.instagram.com/melanomareportgroup,[[https://www.instagram.com/p/Bkfp6z-F6kvYxU6s...
3,https://www.instagram.com/melanomareportgroup,[[https://www.instagram.com/p/BzfD8Zgl0_0AbYpu...
4,https://www.instagram.com/guygonehostile,[[https://www.instagram.com/p/CfWUS-sPhgx9f8Ne...


In [17]:
df2 = df2.explode('Saved data', ignore_index=True)
df2['Post url'] = df2['Saved data'].apply(lambda x: x[0])
df2['Post user'] = df2['Saved data'].apply(lambda x: x[1])

In [8]:
df2['Saved data'][0]

[['https://www.instagram.com/p/B8GDWGhl8ADUAvT-YMw0q1vxVzx8t_UzERsi-k0/',
  'https://www.instagram.com/guygonehostile'],
 ['https://www.instagram.com/p/B8q7rzoFlEwqri0-1n-G67ByulE9kdjzw2NZz00/',
  'https://www.instagram.com/guygonehostile']]

In [19]:
df2.to_csv('test_output_csv_files/saved_output.csv',index=False)

In [18]:
df2 = df2.drop('Saved data',axis=1).drop_duplicates()
df2

,Channel,Post url,Post user
0,https://www.instagram.com/melanomareportgroup,https://www.instagram.com/p/Bkfp6z-F6kvYxU6snq...,https://www.instagram.com/guygonehostile
1,https://www.instagram.com/melanomareportgroup,https://www.instagram.com/p/BzfD8Zgl0_0AbYpuIa...,https://www.instagram.com/guygonehostile
6,https://www.instagram.com/guygonehostile,https://www.instagram.com/p/CfWUS-sPhgx9f8NeVl...,https://www.instagram.com/harsh.aryaa
7,https://www.instagram.com/guygonehostile,https://www.instagram.com/p/ChXQTfNAYWgAxyGiAN...,https://www.instagram.com/harsh.aryaa


In [12]:
for i in df2['Post url']:
    print(i)

https://www.instagram.com/p/Bkfp6z-F6kvYxU6snqUE35r9l8V-h2z24Y6-gs0/
https://www.instagram.com/p/BzfD8Zgl0_0AbYpuIa132lhG1V1Lmzv-b6SWQU0/
https://www.instagram.com/p/CfWUS-sPhgx9f8NeVl1y2ynvcTzk-J0CwYVTRA0/
https://www.instagram.com/p/ChXQTfNAYWgAxyGiANykny3uVNJIC0ra8StAcI0/


In [12]:
df2.explode??

In [21]:
pd.concat??

In [20]:
df2['post url'][1]

'https://www.instagram.com/p/B8q7rzoFlEwqri0-1n-G67ByulE9kdjzw2NZz00/'

In [9]:
credentials = pd.read_csv('test_input_csv_files/Msg_input.csv')

########################################## Scrape ##########################################
df3 = []

try:
    # open browser
    #driver = open_driver()

    # get instagram login credentials
    username = credentials.iloc[0,0]
    password = credentials.iloc[0,1]

    # login
    #driver = login(driver, username, password)

    # open inbox
    driver.get('https://www.instagram.com/direct/inbox/')
    random_time_delay()

    # Infinite scrolling

    # Get seen chats only
    chats = filter_chats(driver)

    # Get last message in the seen chats
    df = get_last_msg(driver, chats)
    df['Login Username'] = username
    print(df.head(5).to_string())

    df3.append(df)

    print('*'*100)

except Exception as e:
    print(e)


df4 = pd.DataFrame(df3)
print(df4.head())

#df4.to_csv('test_output_csv_files/Msg_output.csv',index=False)

	Total Chats: 10
	Selected Chats: 9
['Harsh Arya\nActive now', 'You']
['Mehul Goel\nActive 50m ago', 'More']
['Trisha Rai', 'ok sure n-e-r-d']
['Shourya Kuchhal', 'woah!!🔥']
['ankit panda', 'konsi website haii ye?']


KeyboardInterrupt: 

In [5]:
df3

[]

In [7]:
driver.find_element_by_css_selector('button[class="_acan _acao _acaq _acat"]').text

'Shourya Kuchhal'

In [11]:

user = 'Trisha Rai'
user = user.split('\n')[0]
user

'Trisha Rai'

In [12]:
df =[['Harsh Arya\nActive now', 'You'],
['Mehul Goel\nActive 50m ago', 'More'],
['Trisha Rai', 'ok sure n-e-r-d'],
['Shourya Kuchhal', 'woah!!🔥'],
['ankit panda', 'konsi website haii ye?']]

In [13]:
df2 = pd.DataFrame(df,columns = ['User','Last Message'])
df2

,User,Last Message
0,Harsh Arya\nActive now,You
1,Mehul Goel\nActive 50m ago,More
2,Trisha Rai,ok sure n-e-r-d
3,Shourya Kuchhal,woah!!🔥
4,ankit panda,konsi website haii ye?


In [14]:
'woah!! 🔥'.split()

['woah!!', '🔥']

In [1]:
from fake_useragent import UserAgent
from selenium import webdriver
from selenium.webdriver import Chrome
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager
import bs4
import pandas as pd
import time
import undetected_chromedriver as uc
import argparse
import random
import urllib3
import warnings
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
warnings.filterwarnings("ignore")

ModuleNotFoundError: No module named 'fake_useragent'

In [2]:
credentials = pd.read_csv('test_input_csv_files/Msg_input2.csv')

In [3]:
def random_time_delay():
    time.sleep(random.uniform(60,80))


def open_driver(): 
    ua = UserAgent()
    chrome_options = uc.ChromeOptions()
    chrome_options.add_argument("--window-size=1920,1080")
    chrome_options.add_argument("--disable-extensions")
    chrome_options.add_argument('--disable-notifications')
    chrome_options.add_argument("--mute-audio")
    chrome_options.add_argument("--start-maximized")
    chrome_options.add_argument('--disable-gpu')
    chrome_options.add_argument('--disable-dev-shm-usage')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--ignore-certificate-errors')
    #chrome_options.add_argument(f"user-agent={ua['google chrome']}")

    driver = uc.Chrome(executable_path=ChromeDriverManager().install(), options=chrome_options)
    
    return driver


def login(driver, username, password):  
    # go to login page
    driver.get("https://www.instagram.com")
    random_time_delay()

    # Type username
    emailid = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.NAME, "username"))) 
    emailid.send_keys(username)
    random_time_delay()

    # Type password
    passwordid = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.NAME, "password")))
    passwordid.send_keys(password)
    random_time_delay()

    # Click on the signin button
    signinButton = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH, '//*[@id="loginForm"]/div/div[3]/button/div')))
    ActionChains(driver).move_to_element(signinButton).click().perform()
    random_time_delay()
    
    return driver


def filter_chats(driver):
    all_chats = driver.find_elements_by_css_selector('div[class=" _ab8s _ab8w  _ab94 _ab99 _ab9f _ab9m _ab9p _abcm"]')
    print('\tTotal Chats:',len(all_chats)) 
    result = list(filter(lambda i: i.find_elements_by_css_selector('div[class="_ab8w  _ab94 _ab97 _ab9h _ab9k _ab9p  _abb0 _abcm"]')==[], all_chats)) # unseen blue dot
    print('\tSelected Chats:', len(result))
    return result



def get_last_msg(driver, chats, word_count):
    df = []
    
    for i in chats:
        try:
            # Click on the chat
            i.click()
            random_time_delay()

            # Get the username
            user = driver.find_element_by_css_selector('button[class="_acan _acao _acaq _acat"]').text.split('\n')[0].strip()
            
            # Get last message
            source = driver.page_source
            soup = bs4.BeautifulSoup(source, 'html.parser')
            msgs = [i.text for i in soup.find_all('div',{'class':'_aacl _aaco _aacu _aacx _aad6 _aade'})]
            last_msg = msgs[-1]

            # Go back to the main inbox page
            driver.back()
            random_time_delay()

            row = [user,last_msg]
            print(row)

            if(len(last_msg.split())>word_count):
                df.append(row)


        except Exception as e:
            print(e)
            #print('Non-text modal encountered')
            pass

        
    df2 = pd.DataFrame(df,columns = ['User','Last Message'])
    
    return df2

In [3]:
driver = open_driver()

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 106.0.5249
[WDM] - Get LATEST chromedriver version for 106.0.5249 google-chrome
[WDM] - Driver [C:\Users\aparg\.wdm\drivers\chromedriver\win32\106.0.5249.61\chromedriver.exe] found in cache


In [5]:
driver.current_url

'https://www.instagram.com/p/B8q7rzoFlEwqri0-1n-G67ByulE9kdjzw2NZz00/'

In [6]:
driver.find_element_by_xpath('//*[@id]/div/div/div/div[2]/div/div/div[1]/div/div[3]/div/div/div/div/div[2]/div/article/div/div[2]/div/div/div[2]/section[1]/span[4]/div/div/button/div/svg/path')

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id]/div/div/div/div[2]/div/div/div[1]/div/div[3]/div/div/div/div/div[2]/div/article/div/div[2]/div/div/div[2]/section[1]/span[4]/div/div/button/div/svg/path"}
  (Session info: chrome=106.0.5249.91)
Stacktrace:
Backtrace:
	Ordinal0 [0x01031ED3+2236115]
	Ordinal0 [0x00FC92F1+1807089]
	Ordinal0 [0x00ED66FD+812797]
	Ordinal0 [0x00F055DF+1005023]
	Ordinal0 [0x00F057CB+1005515]
	Ordinal0 [0x00F37632+1209906]
	Ordinal0 [0x00F21AD4+1120980]
	Ordinal0 [0x00F359E2+1202658]
	Ordinal0 [0x00F218A6+1120422]
	Ordinal0 [0x00EFA73D+960317]
	Ordinal0 [0x00EFB71F+964383]
	GetHandleVerifier [0x012DE7E2+2743074]
	GetHandleVerifier [0x012D08D4+2685972]
	GetHandleVerifier [0x010C2BAA+532202]
	GetHandleVerifier [0x010C1990+527568]
	Ordinal0 [0x00FD080C+1837068]
	Ordinal0 [0x00FD4CD8+1854680]
	Ordinal0 [0x00FD4DC5+1854917]
	Ordinal0 [0x00FDED64+1895780]
	BaseThreadInitThunk [0x75FB6739+25]
	RtlGetFull

In [8]:
driver.find_element_by_css_selector('svg[aria-label="Remove"]').click()

In [4]:
df3 = []

for i in range(len(credentials)):
    print('Processing Channel:', i+1, '/', len(credentials))

    try:
        # open browser
        driver = open_driver()

        # get instagram login credentials
        username = credentials.iloc[i,0]
        password = credentials.iloc[i,1]

        # login
        driver = login(driver, username, password)

        # open inbox
        driver.get('https://www.instagram.com/direct/inbox/')
        random_time_delay()

        # Infinite scrolling

        # Get seen chats only
        chats = filter_chats(driver)

        # Get last message in the seen chats
        df = get_last_msg(driver, chats, 10)
        df['Login Username'] = username
        print(df.head(5).to_string())

        df3.append(df)

        # close browsers
        driver.quit()
        random_time_delay()

        print('*'*100)

    except Exception as e:
        print(e)
        continue  


df4 = pd.DataFrame(df3)
print(df4.head())

df4.to_csv('test_output_csv_files/Msg_output.csv',index=False)

Processing Channel: 1 / 1



[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 105.0.5195
[WDM] - Get LATEST chromedriver version for 105.0.5195 google-chrome
[WDM] - Driver [C:\Users\aparg\.wdm\drivers\chromedriver\win32\105.0.5195.52\chromedriver.exe] found in cache


	Total Chats: 10
	Selected Chats: 10
['✨ 𝒟𝒾𝓎𝒶 ✨', 'Here’s my detailsContact number: 8384034120Email : diya9976@gmail.com']
['Shivali Oswal🐅', 'We’ll reach out at an opportune time Shivali']
list index out of range
['Vaishali', '8130904445']
list index out of range
list index out of range
list index out of range
['Shruti Chugh Malik', 'Okay ji']
list index out of range
['Aditi', 'Alright']
Empty DataFrame
Columns: [User, Last Message, Login Username]
Index: []
****************************************************************************************************


ValueError: Must pass 2-d input. shape=(1, 0, 3)

In [6]:
df3

[Empty DataFrame
 Columns: [User, Last Message, Login Username]
 Index: []]

In [2]:
import pandas as pd

In [8]:
df = [['✨ 𝒟𝒾𝓎𝒶 ✨', 'Here’s my detailsContact number: 8384034120Email : diya9976@gmail.com'],
['Shivali Oswal🐅', 'We’ll reach out at an opportune time Shivali']]

In [9]:
df=pd.DataFrame(df,columns = ['User','Last Message'])

In [10]:
df

,User,Last Message
0,✨ 𝒟𝒾𝓎𝒶 ✨,Here’s my detailsContact number: 8384034120Ema...
1,Shivali Oswal🐅,We’ll reach out at an opportune time Shivali


In [11]:
df2 = df.copy()

In [12]:
df2

,User,Last Message
0,✨ 𝒟𝒾𝓎𝒶 ✨,Here’s my detailsContact number: 8384034120Ema...
1,Shivali Oswal🐅,We’ll reach out at an opportune time Shivali


In [13]:
df3 = []

In [15]:
df3.append(df2)

In [16]:
df3

[             User                                       Last Message
 0        ✨ 𝒟𝒾𝓎𝒶 ✨  Here’s my detailsContact number: 8384034120Ema...
 1  Shivali Oswal🐅       We’ll reach out at an opportune time Shivali,
              User                                       Last Message
 0        ✨ 𝒟𝒾𝓎𝒶 ✨  Here’s my detailsContact number: 8384034120Ema...
 1  Shivali Oswal🐅       We’ll reach out at an opportune time Shivali]

In [17]:
pd.DataFrame(df3)

ValueError: Must pass 2-d input. shape=(2, 2, 2)

In [18]:
pd.concat(df3)

,User,Last Message
0,✨ 𝒟𝒾𝓎𝒶 ✨,Here’s my detailsContact number: 8384034120Ema...
1,Shivali Oswal🐅,We’ll reach out at an opportune time Shivali
0,✨ 𝒟𝒾𝓎𝒶 ✨,Here’s my detailsContact number: 8384034120Ema...
1,Shivali Oswal🐅,We’ll reach out at an opportune time Shivali
